In [2]:
import os
import json
import oracledb
import requests

# Enable thick mode (uses Oracle Instant Client)
oracledb.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_15")

# Database connection details
hostname = "10.0.11.59"
port = 1521
service_name = "RMEDB"
username = "RME_DEV"
password = "PASS21RME"

# Microsoft Teams webhook URL
teams_webhook_url = "https://rowadmodern.webhook.office.com/webhookb2/..."

# Unified base directory for storing files
base_dir = r"D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\13.airflow-spark\07.cash-in-message"

# JSON file for storing the last cash-in details
last_transaction_file = os.path.join(base_dir, "last_cash_in.json")

# Ensure the directory exists
os.makedirs(base_dir, exist_ok=True)

# Helper functions
def read_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            try:
                return json.load(file)
            except json.JSONDecodeError as e:
                print(f"Error reading JSON file: {e}. Creating a new JSON file with current transaction.")
                return {}
    return {}

def write_json(file_path, data):
    print(f"Writing data to JSON file: {file_path}")
    print(f"Data to write: {data}")
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"Data successfully written to {file_path}")

try:
    print("Connecting to the database...")
    # Connect to the database
    dsn = oracledb.makedsn(hostname, port, service_name=service_name)
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connected to the database!")

    # Create a cursor
    cursor = connection.cursor()

    # Query to get the latest cash-in transaction
    query = """
    SELECT TO_CHAR(acr.RECEIPT_DATE, 'YYYY-MM-DD') AS receipt_date,
           TRUNC(acr.amount * NVL(acr.exchange_rate, 1)) AS Recipt_amount_EGP,
           acr.comments
    FROM AR_CASH_RECEIPTS_ALL acr
    WHERE acr.ATTRIBUTE_CATEGORY = '83'
      AND acr.ATTRIBUTE1 IS NOT NULL
      AND acr.ATTRIBUTE2 != 'Manual/Netting'
    ORDER BY acr.RECEIPT_DATE DESC
    FETCH FIRST 1 ROW ONLY
    """
    cursor.execute(query)
    result = cursor.fetchone()

    if result:
        # Extract transaction details
        receipt_date, amount_egp, comment = result
        current_transaction = {"date": receipt_date, "amount": int(amount_egp), "comment": comment.strip()}

        print(f"Fetched transaction from database: {current_transaction}")

        # Read the last transaction from JSON
        last_transaction = read_json(last_transaction_file)

        print(f"Last transaction from file: {last_transaction}")

        # Compare transactions
        if current_transaction != last_transaction:
            print("New transaction found! Preparing to send to Teams...")
            # Construct the message
            message = (
                f"New Cash-In Transaction:\n"
                f"📅 Date: {receipt_date}\n"
                f"💰 Amount: {amount_egp:,} EGP\n"
                f"📝 Comment: {comment}"
            )

            # Send the message to Microsoft Teams
            payload = {"text": message}
            response = requests.post(teams_webhook_url, json=payload, verify=False)

            if response.status_code == 200:
                print("Message sent to Microsoft Teams successfully!")

                # Update the JSON file with the latest transaction
                write_json(last_transaction_file, current_transaction)
                print(f"Updated JSON file: {last_transaction_file}")
            else:
                print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")
        else:
            print("No new cash-in transactions found.")
    else:
        print("No cash-in transactions found in the database.")

except oracledb.Error as error:
    print(f"Error connecting to database: {error}")

finally:
    # Close the cursor and connection
    try:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
    except oracledb.Error as error:
        print(f"Error while closing cursor: {error}")

    try:
        if 'connection' in locals() and connection is not None:
            connection.close()
    except oracledb.Error as error:
        print(f"Error while closing connection: {error}")


Connecting to the database...
Connected to the database!
Fetched transaction from database: {'date': '2025-02-06', 'amount': 3917447, 'comment': 'ipc 39 - ORA ZED BARK ph2.'}
Last transaction from file: {}
New transaction found! Preparing to send to Teams...


c:\Users\Omar Essam2\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rowadmodern.webhook.office.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Failed to send message. Status code: 404, Response: 
